https://www.plattform-i40.de/IP/Redaktion/EN/Downloads/Publikation/Details_of_the_Asset_Administration_Shell_Part1_V3.html

In [ ]:
from rdflib import BNode, ConjunctiveGraph, Dataset, Graph, URIRef

# Construct AAS (template) from OWL ontology

In [ ]:
prefixes = {
    'prov': 'http://www.w3.org/ns/prov#',
    'skos': 'http://www.w3.org/2004/02/skos/core#',
    'mas4ai': 'http://example.org/MAS4AI_GenericModel#',
    'aas': 'https://admin-shell.io/aas/3/0/RC01/',
    'aasenv': 'https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShellEnvironment/',
    'aasaas': 'https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShell/',
    'aassm': 'https://admin-shell.io/aas/3/0/RC01/Submodel/',
    'aassmc': 'https://admin-shell.io/aas/3/0/RC01/SubmodelElementCollection/',
    'aasrefer': 'https://admin-shell.io/aas/3/0/RC01/Referable/',
    'aasprop': 'https://admin-shell.io/aas/3/0/RC01/Property/',
    'aasrefe': 'https://admin-shell.io/aas/3/0/RC01/ReferenceElement/',
    'aasref': 'https://admin-shell.io/aas/3/0/RC01/Reference/',
    'aaskey': 'https://admin-shell.io/aas/3/0/RC01/Key/',
    'aaskeyt': 'https://admin-shell.io/aas/3/0/RC01/KeyType/',
    'aaskind': 'https://admin-shell.io/aas/3/0/RC01/HasKind/',
    'aasmod': 'https://admin-shell.io/aas/3/0/RC01/ModelingKind/',
    'aassem': 'https://admin-shell.io/aas/3/0/RC01/HasSemantics/',

    'aasrel': 'https://admin-shell.io/aas/3/0/RC01/RelationshipElement/',
    'aasdata': 'https://admin-shell.io/aas/3/0/RC01/HasDataSpecification/',
    'aasrange': 'https://admin-shell.io/aas/3/0/RC01/Range/',
    'aasida': 'https://admin-shell.io/aas/3/0/RC01/Identifiable/',
    'aaside': 'https://admin-shell.io/aas/3/0/RC01/Identifier/'
}

def add_prefixes(graph):
    for k,v in prefixes.items():
        graph.namespace_manager.bind(k, URIRef(v))

## Initialize data model/graph

In [ ]:
dataset = Dataset()

file_name = 'mas4aiDEMO'
g_owl = dataset.graph(identifier=URIRef('http://mas4ai.eu/id/graph/owl'))
g_owl.parse(f'examples/{file_name}.ttl')

g_AAS_ont = dataset.graph(identifier=URIRef('https://admin-shell.io/aas/3/0/RC01/'))
g_AAS_ont.parse('https://raw.githubusercontent.com/admin-shell-io/aas-specs/master/schemas/rdf/rdf-ontology.ttl', format='text/turtle')
# g_AAS_ont.parse('https://raw.githubusercontent.com/admin-shell-io/aas-specs/draft-V3RC02-schemas/schemas/rdf/rdf-ontology.ttl', format='text/turtle')

g_AAS = dataset.graph(identifier=URIRef('http://mas4ai.eu/id/graph/aas'))

g_conj = ConjunctiveGraph(dataset.store)

### Enrich the shapes graph

#### Import referenced ontologies

In [ ]:
[o for o in g_owl.objects(predicate=URIRef('http://www.w3.org/2002/07/owl#imports'))]

#### Add statements to indicate which classes should get an AAS

In [ ]:
aas_classes = [
    'http://www.tno.nl/mas4aiDEMO#RoboticArm', 'http://www.tno.nl/mas4aiDEMO#Tool',
]

for c in aas_classes:
    g_owl.add((URIRef(c), URIRef('http://example.org/MAS4AI_GenericModel#hasInterface'), BNode()))

#### Infer properties from super classes
Use inference tool?

In [ ]:
g_owl.update('''
INSERT {
  ?Property rdfs:domain ?SubClass
}
WHERE {
  VALUES ?PropertyType {owl:ObjectProperty owl:DatatypeProperty}
  ?Property a ?PropertyType ;
    rdfs:domain ?Class .
  ?SubClass rdfs:subClassOf+ ?Class .
}
''')

## Construct AAS components
* Construct components, starting from the lowest level.
* Add (temporary) provenance data (`prov:wasDerivedFrom`) that can be used to link the different AAS components.

### Property
* The `sh:path` of a `sh:PropertyShape` should be mapped to the `aassem:semanticId` reference;
* A `sh:PropertyShape` without a `sh:class` is an `aas:Property`.

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Property
  ?Property_iri a aas:Property ;
    aasprop:valueType ?dataType ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Property ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?Property ;
  .
}
WHERE {
  ?Property a owl:DatatypeProperty ; #filter out reference properties
    rdfs:range ?dataType .

  BIND(iri(concat( "http://mas4ai.eu/id/property/template/", struuid() )) as ?Property_iri)
}
''').graph.serialize())

### ReferenceElement
* A `sh:PropertyShape` that has a `sh:class`, is mapped to a `aas:ReferenceElement`.

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Reference Element
  ?ReferenceElement_iri a aas:ReferenceElement ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?ReferenceElement ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?ReferenceElement ;
  .
}
WHERE {
  ?ReferenceElement a owl:ObjectProperty ; #filter on reference properties
      rdfs:range ?Class .

  FILTER EXISTS { ?Class mas4ai:hasInterface [] } #otherwise it should be a Submodel or SMC

  BIND(iri(concat( "http://mas4ai.eu/id/referenceElement/template/", struuid() )) as ?ReferenceElement_iri)
}
''').graph.serialize())

### Submodel Element Collection
* A `sh:NodeShape` of which the related `sh:targetClass` does not have `mas4ai:hasInterface` and is not 'directly related' to a `sh:NodeShape` of this kind, should be mapped to a `aas:SubmodelElementCollection` ('PropertyCollection').
* A `sh:PropertyShape` with `sh:maxCount > 1` should be embedded in an `aas:SubmodelElementCollection` (for example jobs) ;

#### 'Property Collection'

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Class ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?NodeShape ;
  .
}
WHERE {
  {
    ?Class a owl:Class .
  } UNION {
    ?Class a rdfs:Class .
  }

  ?Property rdfs:range ?Class .

  FILTER EXISTS { [] rdfs:domain ?Class }
  FILTER NOT EXISTS { ?Class mas4ai:hasInterface [] }
  FILTER NOT EXISTS {
    #exclude classes 'directly related to' the 'main' MAS4AI class
    ?Property rdfs:domain/mas4ai:hasInterface [] ;
      rdfs:range ?Class .
  }

  BIND(iri(concat( "http://mas4ai.eu/id/smc/template/", struuid() )) as ?SMC_iri)
}
''').graph.serialize())

#### Cardinality >1 properties

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection ;
    aasrefer:idShort ?SMCidShort  ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Property ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?Property, ?Class ;
  .
}
WHERE {
  VALUES ?PropertyType {owl:ObjectProperty owl:DatatypeProperty}
  {
    ?Property a ?PropertyType ;
      rdfs:label ?propertyLabel ;
      owl:maxCardinality ?maxCardinality .
    FILTER( ?maxCardinality > 1 )
  } UNION {
    ?Property a ?PropertyType ;
      rdfs:label ?propertyLabel .
    FILTER NOT EXISTS { ?Property owl:maxCardinality [] } #if owl:maxCardinality>1 or not defined then embed the property in an ElementCollection
  }

  OPTIONAL { ?Property owl:range ?Class }

  FILTER NOT EXISTS {
    #exclude object properties 'directly related to' a 'main' MAS4AI class
    ?Property a owl:ObjectProperty ;
      rdfs:domain/mas4ai:hasInterface [] .
  }
  FILTER NOT EXISTS { ?Property rdfs:range/mas4ai:hasInterface [] }

  BIND(iri(concat( "http://mas4ai.eu/id/smc/template/", struuid() )) as ?SMC_iri)
  BIND(concat(?propertyLabel, 's') as ?SMC_idShort)
}
''').graph.serialize())

#### Cardinality >1 object properties (directly related to main AAS class)

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection ;
    aasrefer:idShort ?SMCidShort  ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Class ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?Property, ?Class ;
  .
}
WHERE {
  {
    ?Property a owl:ObjectProperty ;
      rdfs:label ?propertyLabel ;
      owl:maxCardinality ?maxCardinality .
    FILTER( ?maxCardinality > 1 )
  } UNION {
    ?Property a owl:ObjectProperty ;
      rdfs:label ?propertyLabel .
    FILTER NOT EXISTS { ?Property owl:maxCardinality [] } #if owl:maxCardinality>1 or not defined then embed the property in an ElementCollection
  }

  ?Property rdfs:range ?Class

  FILTER EXISTS {
    #only include classes 'directly related to' the 'main' MAS4AI class
    ?Property a owl:ObjectProperty ;
      rdfs:domain/mas4ai:hasInterface [] .
  }
  FILTER NOT EXISTS { ?Property rdfs:range/mas4ai:hasInterface [] }

  BIND(iri(concat( "http://mas4ai.eu/id/smc/template/", struuid() )) as ?SMC_iri)
  BIND(concat(?propertyLabel, 's') as ?SMC_idShort)
}
''').graph.serialize())

##### Relation between SMCs and properties + reference elements + SMCs

In [ ]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?SMC aassmc:value ?Value .
  }
}
WHERE {
  {
    ?SMC a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom ?Class .

    ?Class a owl:Class .
    ?Property rdfs:domain ?Class .
  } UNION {
    ?SMC a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom ?Property .
  }

  ?SubmodelElementType rdfs:subClassOf+ aas:SubmodelElement .
  ?Value a ?SubmodelElementType ;
    prov:wasDerivedFrom ?Property .

  FILTER (?Value != ?SMC)
}
''')

### Submodel
* A `sh:NodeShape` is converted to an `aas:Submodel` and embeds all `sh:PropertyShape`s that are linked to the `sh:NodeShape` via `sh:property` if they are object properties.

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Submodel (SM)
  ?SM_iri a aas:Submodel ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Property ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?Class, ?Property ;
  .
}
WHERE {
  ?Class a owl:Class .

  {
    # Main Submodel for AAS
    FILTER EXISTS { ?Class mas4ai:hasInterface [] }
    FILTER NOT EXISTS { [] rdfs:subClassOf ?Class }
  } UNION {
    # Submodel for object properties with domain 'main' AAS class
    ?Property a owl:ObjectProperty ;
      rdfs:domain/mas4ai:hasInterface [] ;
      rdfs:range ?Class .
  }

  FILTER EXISTS { [] rdfs:domain ?Class }

  BIND(iri(concat( "http://mas4ai.eu/id/sm/template/", struuid() )) as ?SM_iri)
}
''').graph.serialize())

##### Relation between submodels and properties + submodel element collections

In [ ]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Submodel aassm:submodelElements ?SubmodelElement .
  }
}
WHERE {
  {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom ?Class .

    ?Property a owl:DatatypeProperty ;
      rdfs:domain ?Class .

    ?SubmodelElement prov:wasDerivedFrom ?Property .

    FILTER NOT EXISTS { ?Submodel prov:wasDerivedFrom/rdfs:range/mas4ai:hasInterface [] }
  } UNION {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom ?Property .
    ?SubmodelElement a aas:ReferenceElement ;
      prov:wasDerivedFrom ?Property .
  } UNION {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom ?Property .

    ?SubmodelElement a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom ?Property .
    FILTER NOT EXISTS { ?Property owl:maxCardinality 1 }
  }

  #exclude nested submodel elements
  FILTER NOT EXISTS {
    [] a aas:SubmodelElementCollection ;
      aassmc:value ?SubmodelElement .
  }
}
''')

### AssetAdministrationShell

In [ ]:
add_prefixes(dataset)

g_AAS.parse(data=g_owl.query('''
CONSTRUCT {
  # Asset Administration Shell (AAS)
  ?AAS_iri a aas:AssetAdministrationShell ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:keys [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?AASClass ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?AASClass ;
  .
}
WHERE {
  {
    ?AASClass a owl:Class .
  } UNION {
    ?AASClass a rdfs:Class .
  }

  ?AASClass mas4ai:hasInterface [] .

  BIND(iri(concat( "http://mas4ai.eu/id/aas/template/", struuid() )) as ?AAS_iri)
}
''').graph.serialize())

##### Relations between asset administration shell and submodels

In [ ]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?AAS aasaas:submodels ?Submodel .
  }
}
WHERE {
  {
    # Main Submodel for AAS
    ?AAS a aas:AssetAdministrationShell ;
      prov:wasDerivedFrom ?AASClass .

    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom ?AASClass .

    FILTER NOT EXISTS { ?Submodel prov:wasDerivedFrom/a owl:ObjectProperty }
  } UNION {
    ?AAS a aas:AssetAdministrationShell ;
      prov:wasDerivedFrom ?AASClass .

    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom ?Class, ?Property .

    ?Property a owl:ObjectProperty ;
      rdfs:domain ?AASClass ;
      rdfs:range ?Class .
  }

  ?AASClass mas4ai:hasInterface [] .
}
''')

##### Reference element value (reference to other AAS)

In [ ]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?ReferenceElement aasrefe:value [
        a aas:Reference ;
        aasref:keys [
          a aas:Key ;
          aaskey:idType aaskeyt:AssetAdministrationShell ;
          aaskey:value ?_ReferenceAAS ;
        ] ;
      ] ;
    .
  }
}
WHERE {
  ?ReferenceElement a aas:ReferenceElement ;
    prov:wasDerivedFrom ?Property .

  ?Property a owl:ObjectProperty .

  ?ReferenceAAS a aas:AssetAdministrationShell ;
    prov:wasDerivedFrom ?Class .

  FILTER EXISTS {?Class mas4ai:hasInterface []}

  BIND(str(?ReferenceAAS) as ?_ReferenceAAS)
}
''')

### AssetAdministrationShellEnvironment

In [ ]:
add_prefixes(dataset)

g_AAS.update('''
INSERT {
  <http://mas4ai.eu/def/WP4/AASEnv> a aas:AssetAdministrationShellEnvironment ;
    aasenv:assetAdministrationShells ?AAS ;
    aasenv:submodels ?Submodel .
}
WHERE {
  ?AAS a aas:AssetAdministrationShell ;
    aasaas:submodel ?Submodel .

  ?Submodel a aas:Submodel .
}
''')

### Add common statements for objects of a certain type
(if they don't exist yet)

#### HasKind
`aaskind:kind`

In [ ]:
g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Object aaskind:kind aasmod:Template .
  }
}
WHERE {
  ?Object a/rdfs:subClassOf* aas:HasKind .
  FILTER NOT EXISTS { ?Object aaskind:kind [] }
}
''')

#### Referable
|Shapes predicate|AAS predicate|
|---|---|
| `rdfs:label` | `aasrefer:idShort`     |
| `rdfs:comment` | `aasrefer:description` |
| `skos:prefLabel` | `aasrefer:displayName` |

In [ ]:
g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Object aasrefer:idShort ?idShort ;
      rdfs:label ?label ;
      aasrefer:description ?description ;
      aasrefer:displayName ?displayName ;
    .
  }
}
WHERE {
  # Prefer attributes from owl:Class
  { SELECT ?Object (if(bound(?Class), ?Class, ?Property) as ?Resource) {
    ?Object a/rdfs:subClassOf* aas:Referable ;
    OPTIONAL {
      ?Object prov:wasDerivedFrom ?Class .
      { ?Class a owl:Class } UNION { ?Class a rdfs:Class }
    }
    OPTIONAL {
      ?Object prov:wasDerivedFrom ?Property .
      { ?Property a owl:DatatypeProperty } UNION { ?Property a owl:ObjectProperty }
    }
  } }

  ?Resource rdfs:label ?label .
  OPTIONAL { ?Object aasrefer:idShort ?_idShort }
  BIND ( REPLACE(COALESCE(?_idShort, ?label), " ", "_") AS ?idShort )

  OPTIONAL {
    ?Resource rdfs:comment ?comment .
    OPTIONAL { ?Object aasrefer:description ?_description }
    BIND ( COALESCE(?_description, ?comment) AS ?description )
  }

  OPTIONAL {
    ?Resource skos:prefLabel ?prefLabel .
    OPTIONAL { ?Object aasrefer:displayName ?_displayName }
    BIND ( COALESCE(?_displayName, ?prefLabel) AS ?displayName )
  }
}
''')

## Inspect AAS graph

In [ ]:
print(g_AAS.serialize())

In [ ]:
# Store to file
g_AAS.serialize(f'examples/{file_name}_template.aas.ttl')
# g_AAS.serialize('examples/f'examples/{file_name}_template.aas.jsonld', format='json-ld')